# Replica project

### The data

In [14]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

from glob import glob
from scipy import sparse

from torch.utils.data import DataLoader

from utils import *
from metrics import *
from dataloader_replica import ReplicaDataset
from train_replica import train_replica
from model_replica import ReplicaNet

import umap
import numpy as np

from utils import * 
path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = pd.read_csv(data_dir + 'full_data.csv').drop(columns=['Unnamed: 0', 'level_0'])
print(data.shape)
data.head()

(6163, 45)


,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
0,002ca9c06a704c779e69d93cb4fadaa3,649,45588b4e33444ed8b34c523bac808bc4,158B,556,TIZIANO Vecellio (copia da),S.Mariá Maddalena. cm. 123 x 91,TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158B%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50.0,train
1,0038ab462149452b98a4d7359a06f9f4,1571,ae828b07e4284d80b652206ffb77278a,158C,120,TIZIANO Vecellio (copia da),"Morte di S. Pietro Martire. mm. 123,8 x 84,2",TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158C%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51.0,test
2,00474897de044f7fb1ef16888dc198f3,1107,76b2c4aeab404bc79768117e3ddc62b1,47A,858,BELLINI Giovanni (cerchia di),Pietà.,"BELLINI, Giovanni (cerchia di -)","Bellini, Giovanni",ulan:500019244,...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/47A%2F47A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498.0,test
3,009c03c265b8418495da038bc3826bf7,1764,c5839e5220e5419e92d878396b0c04bb,135C,57,SEC XVI ex,Un soffitto. Part.: Leda e il cigno.,SEC. XVI ex.,NaN,NaN,...,NaN,NaN,https://dhlabsrv4.epfl.ch/iiif_cini/135C%2F135...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,450ee2aceb014709b23a93e1163937c5,c5839e5220e5419e92d878396b0c04bb,POSITIVE,2018-05-07 08:28:11.797723+00:00,92.0,train
4,00aeddb93a8a4120aa984eda6c822747,747,4d02860f58f2443ba4d1ef088756f854,21C,228,BASSANO Leandro,"La preparazione del pranzo. cm. 88,3 x 102,9","BASSANO, Leandro.","Bassano, Leandro",ulan:500015945,...,Bassano del Grappa,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,4d02860f58f2443ba4d1ef088756f854,27bad0f213a342c3987e8cec61e40550,POSITIVE,2018-03-23 15:41:53.094226+00:00,218.0,train


### Creating embeddings of all the images

In [22]:
def create_model(model_name, pooling):
    if model_name == "resnet50":
        model = models.resnet50(pretrained=True)
    elif model_name == "efficientnet0":
        model = models.efficientnet_b0(pretrained=True)
    elif model_name == "efficientnet7":
        model = models.efficientnet_b7(pretrained=True)

    if pooling == "avg":
        newmodel = torch.nn.Sequential(
            *(list(model.children())[:-2]), nn.AdaptiveAvgPool2d((1, 1))
        )
    elif pooling == 'max':
        newmodel = torch.nn.Sequential(
            *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1, 1), )
        )
    
    return newmodel

def preprocess_image_orig(img_name, resolution=480):
    img = Image.open(img_name)
    tfms = transforms.Compose(
        [
            transforms.Resize((resolution, resolution)),
            transforms.ToTensor(),
            #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    return tfms(img).unsqueeze(0)


def get_embedding_orig(img, model, device='cpu'):
    embedding = model(img.squeeze(1).to(device))[0].cpu().detach().numpy()
    norm = np.linalg.norm(embedding)
    return embedding / norm


# print(model)


In [26]:
img = preprocess_image_orig(data_dir + 'subset/' + data['uid'][0] + '.jpg')
newmodel = create_model('resnet50', 'max')
embedding = get_embedding_orig(img, newmodel).squeeze(1).squeeze(1)
print(embedding.shape)
embedding

(2048,)


array([0.01368189, 0.02313367, 0.01797748, ..., 0.02025607, 0.01675818,
       0.01065022], dtype=float32)

In [27]:
newmodel = create_model('resnet50', 'max')
embeddings = [[uid, get_embedding_orig(preprocess_image_orig(data_dir + 'subset/' + uid + '.jpg'), newmodel).squeeze(1).squeeze(1)] for uid in tqdm(data['uid'].unique())]

100%|██████████| 4357/4357 [23:13<00:00,  3.13it/s]


In [ ]:
np.save('resnet50_max_480.npy', np.array(embeddings, dtype=np.ndarray))

In [7]:
embeddings = np.load(data_dir + 'resnet50_max_240.npy', allow_pickle=True)
embeddings

array([['45588b4e33444ed8b34c523bac808bc4',
        array([0.01273958, 0.02077351, 0.02422727, ..., 0.01849872, 0.02054269,
       0.01348173], dtype=float32)],
       ['ae828b07e4284d80b652206ffb77278a',
        array([0.01653748, 0.02765266, 0.02819748, ..., 0.01199384, 0.01694367,
       0.01413397], dtype=float32)],
       ['76b2c4aeab404bc79768117e3ddc62b1',
        array([0.01377614, 0.02740756, 0.02341555, ..., 0.01013662, 0.01947277,
       0.01492655], dtype=float32)],
       ...,
       ['ffd362f3b0f64665a6d4708b62c6333c',
        array([0.01821662, 0.01876744, 0.0185441 , ..., 0.01633258, 0.02021206,
       0.01388135], dtype=float32)],
       ['ffd9ce3649364906810269748b8a5ddc',
        array([0.01877906, 0.0240321 , 0.02262984, ..., 0.01359423, 0.01878336,
       0.01530929], dtype=float32)],
       ['fffb47ce9194475087f2c7dc1eaaa41a',
        array([0.02208385, 0.02518187, 0.02769189, ..., 0.01206282, 0.02445544,
       0.01236099], dtype=float32)]], dtype=object)

### Finding most similar embeddings

In [3]:
list_downloaded = [
    file.split("/")[-1].split(".")[0] for file in glob(data_dir + "subset/*")
]
print(len(list_downloaded))

train_test = data[data["set"].notnull()].reset_index() #[data['img1'].isin(list_downloaded)][data['img2'].isin(list_downloaded)]
print(train_test.shape)


4593
(2862, 46)


In [4]:
def get_scores(embeddings, train_test, data, list_downloaded):
    tree = make_tree_orig(embeddings)
    Cs = []
    Bs = []
    pos = []
    ranks = []

    for i in tqdm(range(train_test.shape[0])):
        if (train_test["img1"][i] in list_downloaded) and (train_test["img2"][i] in list_downloaded):
            list_theo = (
                list(train_test[train_test["img1"] == train_test["uid"][i]]["img2"])
                + list(train_test[train_test["img2"] == train_test["uid"][i]]["img1"])
                + [train_test["uid"][i]]
            )
            Bs.append(list_theo)
            list_sim = find_most_similar_orig(
                train_test["uid"][i], tree, embeddings, list(data["uid"].unique()), n=4357
            )
            Cs.append(list_sim[:400])
            matches = find_pos_matches(list_sim[:400], list_theo, how="all")
            pos.append(matches)
            rank = make_rank(list_sim, list_theo)
            ranks.append(rank)
            
    posses = [po for p in pos for po in p]
    posses_min = [p[0] for p in pos]
    posses_med = [np.median(np.array(p)) for p in pos]

    mean_position = np.mean(np.array(posses))
    mean_min_position = np.mean(np.array(posses_min))
    mean_median_position = np.mean(np.array(posses_med))
            
    print('all positions', mean_position)
    print('min positions', mean_min_position)
    print('median positions', mean_median_position)

    map = mean_average_precision(ranks)
    print('mean average precision', map)

    recall_400 = np.mean([recall_at_k(ranks[i], 400) for i in range(len(ranks))])
    recall_200 = np.mean([recall_at_k(ranks[i], 200) for i in range(len(ranks))])
    recall_100 = np.mean([recall_at_k(ranks[i], 100) for i in range(len(ranks))])
    recall_50 = np.mean([recall_at_k(ranks[i], 50) for i in range(len(ranks))])
    recall_20 = np.mean([recall_at_k(ranks[i], 20) for i in range(len(ranks))])
    print('recall @ 400', recall_400)
    print('recall @ 200', recall_200)
    print('recall @ 100', recall_100)
    print('recall @ 50', recall_50)
    print('recall @ 20', recall_20)

    return mean_position, mean_min_position, mean_median_position, map, recall_400, recall_200, recall_100, recall_50, recall_20


In [5]:
from sklearn.decomposition import PCA, TruncatedSVD

def get_lower_dimension_(embeddings, dimensions=100, method='umap'):
    if method == 'umap':
        embeddings_new = umap.UMAP(n_components=dimensions, metric='cosine').fit(np.vstack(embeddings[:,1]))
        embeddings_new = embeddings_new.embedding_
    elif method == 'pca':
        embeddings_new = PCA(n_components=dimensions).fit_transform(np.vstack(embeddings[:,1]))
    elif method == 'svd':
        embeddings_new = TruncatedSVD(n_components=dimensions).fit_transform(np.vstack(embeddings[:,1]))
    
    return embeddings_new


In [9]:

for model in ['resnext-101',]: #, "efficientnet0", "efficientnet7" 'densenet161',==3 dimensions?? 'regnet_x_32gf', 'resnet101', 'resnet152',
    for pool in ["avg"]: #, "max"
        for resolution in ["480"]: #"240", 
            print(model, pool, resolution)
            embeddings = np.load(
                data_dir + model + "_" + pool + "_" + resolution + ".npy",
                allow_pickle=True,
            )        
            embs = get_lower_dimension_(embeddings, dimensions=500, method='pca')
            embeddings_new = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
            #embs = get_lower_dimension_(embeddings, dimensions=500, method='svd')
            #embeddings_new_2 = np.array([[embeddings[i, 0], embs[i]] for i in range(len(embs))], dtype=np.ndarray)
            print('original scores')
            get_scores(embeddings, train_test, data, list_downloaded)
            print('dimensionality reduction scores 500')
            get_scores(embeddings_new, train_test, data, list_downloaded)
            print('\n\n\n\n')
            #print('dimensionality reduction scores 250')
            #get_scores(embeddings_new_2, train_test, data, list_downloaded)
            
            

resnext-101 avg 240
original scores


100%|██████████| 2862/2862 [00:27<00:00, 105.17it/s]


all positions 294.01240625867973
min positions 134.63024217365623
median positions 334.555522740697
mean average precision 0.10641506213192588
recall @ 400 0.5087109385987117
recall @ 200 0.4093215293599228
recall @ 100 0.3380469533747738
recall @ 50 0.2656576097657019
recall @ 20 0.18377750588323533
dimensionality reduction scores 500


100%|██████████| 2862/2862 [00:11<00:00, 254.34it/s]


all positions 293.94343116378116
min positions 137.32132309509745
median positions 333.22947430596577
mean average precision 0.1042114731328412
recall @ 400 0.5147336371051668
recall @ 200 0.40461494107388973
recall @ 100 0.33125310569197164
recall @ 50 0.25854991139988187
recall @ 20 0.17897762964213051





resnext-101 avg 480
original scores


...

all positions 309.1141560966577
min positions 146.92439456585942
median positions 353.6848789131719
mean average precision 0.07963686422206237
recall @ 400 0.4465014672929617
recall @ 200 0.34974287214393535
recall @ 100 0.27523345427952634
recall @ 50 0.21389896773830622
recall @ 20 0.1539591126862243





resnet152 avg 480
original scores


100%|██████████| 2862/2862 [00:27<00:00, 105.50it/s]


all positions 302.46440144431074
min positions 144.35144713526284
median positions 345.3449497932664
mean average precision 0.08735369381220447
recall @ 400 0.4808108082768449
recall @ 200 0.37789802079524465
recall @ 100 0.2901960922191283
recall @ 50 0.22923241357972604
recall @ 20 0.1622933835869453
dimensionality reduction scores 500


100%|██████████| 2862/2862 [00:11<00:00, 255.78it/s]


all positions 298.8769558374225
min positions 142.38216184288245
median positions 343.2028942705257
mean average precision 0.08805038763972971
recall @ 400 0.49518770099100873
recall @ 200 0.3879733543348428
recall @ 100 0.3025815918019108
recall @ 50 0.22045936114158202
recall @ 20 0.15933535847889066







In [6]:
for model in ['resnext-101',]: #, "efficientnet0", "efficientnet7" 'densenet161',==3 dimensions?? 'regnet_x_32gf', 'resnet101', 'resnet152',
    for pool in ["avg"]: #, "max"
        for resolution in ["480"]: #"240", 
            print(model, pool, resolution)
            embeddings = np.load(
                data_dir + model + "_" + pool + "_" + resolution + ".npy",
                allow_pickle=True,
            )        
            print('original scores')
            get_scores(embeddings, train_test, data, list_downloaded)
            

resnext-101 avg 480
original scores


100%|██████████| 2862/2862 [00:34<00:00, 82.71it/s] 


all positions 286.7803907045644
min positions 126.7005316007088
median positions 327.55197873597166
mean average precision 0.12735763379073267
recall @ 400 0.5292476490497754
recall @ 200 0.4385745694221772
recall @ 100 0.36267661425664965
recall @ 50 0.2844490385246439
recall @ 20 0.2171340440094132


In [15]:
def make_training_set_orig(embeddings, train_test, data_dir):
    tree = make_tree_orig(embeddings)

    
    Cs = []
    for i in tqdm(range(train_test.shape[0])):
            list_theo = (
                list(train_test[train_test["img1"] == train_test["uid"][i]]["img2"])
                + list(train_test[train_test["img2"] == train_test["uid"][i]]["img1"])
                + [train_test["uid"][i]]
            )
            list_sim = find_most_similar_no_theo(
                train_test["uid"][i], tree, embeddings, list(data["uid"].unique()), list_theo, n=10
            )
            Cs.append(list_sim)
            
    print(len(Cs))

    train_test['C'] = Cs

    list_downloaded = [file.split('/')[-1].split('.')[0] for file in glob(data_dir + 'subset/*')]


    train_test = train_test[train_test['img1'].isin(list_downloaded)]
    train_test = train_test[train_test['img2'].isin(list_downloaded)]

    final = train_test[['img1', 'img2', 'C', 'set']].explode('C')
    final.columns = ['A', 'B', 'C', 'set']
    print(final.shape)

    final[final['set'] == 'train'].reset_index().to_csv(data_dir + 'abc_train.csv')
    final[final['set'] == 'test'].reset_index().to_csv(data_dir + 'abc_test.csv')

    return final


In [17]:
final = make_training_set_orig(embeddings, train_test, data_dir)
final.sample(5)

100%|██████████| 2862/2862 [00:44<00:00, 63.90it/s]


2862
(14096, 4)


,A,B,C,set
952,89dd078005de443382875fcc00c78fac,36e4b7aba6f046de916cfae3d999369c,ea158e43a0184c3582a870fc9db39097,train
2502,e7db4731e0a34029832505f7a6f4ccb1,e05ca03f3fc44d5a91716745de81fc34,8ac721cae3184a4eb3665ea387d38cbd,train
849,d25678d4d71e4f9cbbc15bbb155f3ea2,b9e8f6bd9c544c41b2196be93c3352a7,8ae58e27ec284e84bc191b0995df5e02,test
2366,ea75a7a900dd44818f0fa4fe55fe606b,a57d5a95036a40e49b9715adba79ac32,0060668f7bb54ee3b76923bf482a8b6b,test
2575,7ee9e023d1854a1da4ab1588ee94947e,d3fba7a263e540efa87af9ca19c58917,0f21c000cfaf45d8ba860654a4bd32c5,train


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])